# Tutorial 1: Density estimation using the P1-Mono dataset

## Introduction

This tutorial shows how to compute the log-probability density function of the single-cell flow cytometry data.

In this example we use the P1-Mono dataset (differentiating monocytes of healthy patient P1). Data have been pre-processed with the logicle transformation. Marker expression was scaled to [0,1] with the min-max transform.

In [9]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.model_selection import ParameterGrid
import os
import sys

sys.path.append(os.path.abspath("../src/TimeFlow/"))

import dataset
from dataset import CytometryData
import density
from density import DensityEstimation
import train
from train import Training

In [10]:
np.random.seed(1)
torch.manual_seed(1)

In [12]:
# Load the CSV file with pre-processed and scaled data
data = pd.read_csv("../Pre-processed-datasets/P1_monocytes.csv")
# We exclude the cell labels, which are not used during density estimation and pseudotime computation
data = data.iloc[:,:20]

In [13]:
data

,CD200,CD14,CD45,CD45RA,CD64,CD3,CD15,CD133,CD117,CD56,HLA.DR,CD19,CD33,CD34,CD371,CD7,CD16,CD123,CD36,CD38
0,0.476360,0.210000,0.380798,0.593579,0.209033,0.556763,0.390368,0.760927,0.811495,0.433740,0.547981,0.682992,0.641786,0.786273,0.715657,0.572932,0.262740,0.470331,0.275481,0.872200
1,0.408532,0.240234,0.612831,0.502399,0.276553,0.566520,0.492926,0.237612,0.413990,0.322411,0.320588,0.551182,0.656294,0.242648,0.744157,0.618874,0.218125,0.983270,0.455801,0.687064
2,0.322921,0.334046,0.493350,0.395761,0.166498,0.595043,0.401960,0.436152,0.416024,0.333763,0.392882,0.446213,0.570805,0.296832,0.571086,0.339374,0.342555,0.954020,0.413528,0.767273
3,0.404659,0.341854,0.466869,0.505488,0.875529,0.654263,0.551584,0.455843,0.422862,0.440552,0.576057,0.648720,0.893499,0.105882,0.700903,0.817287,0.160520,0.676352,0.433278,0.827099
4,0.359422,0.737795,0.704663,0.609532,0.781949,0.596656,0.527768,0.598056,0.424379,0.388737,0.449520,0.745688,0.930050,0.119879,0.841939,0.485037,0.359423,0.591350,0.886581,0.637507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39251,0.303462,0.290607,0.249948,0.557437,0.469470,0.761341,0.378994,0.480666,0.437503,0.326104,0.739206,0.501915,0.776038,0.224030,0.846944,0.779584,0.199911,0.736472,0.565429,0.815701
39252,0.322432,0.295687,0.394565,0.411515,0.285041,0.733938,0.312513,0.590894,0.625841,0.315181,0.466644,0.577548,0.508150,0.745892,0.292928,0.659374,0.179430,0.713749,0.652844,0.841941
39253,0.472677,0.314547,0.409916,0.413875,0.290130,0.695890,0.337164,0.849519,0.708099,0.303322,0.322242,0.331928,0.502452,0.788373,0.262349,0.527730,0.256492,0.389099,0.345958,0.513077
39254,0.489425,0.357431,0.632018,0.619991,0.479557,0.700178,0.380154,0.488515,0.435385,0.277057,0.704551,0.532311,0.864917,0.178811,0.892551,0.896200,0.164894,0.664771,0.760477,0.775643


In [14]:
# Create dataset instances
# Training set
dataset_train = CytometryData(data, mode='train')
# Validation set
dataset_val = CytometryData(data, mode='val')     
# Test set
# dataset_test = CytometryData(data, mode='test')  

In [15]:
# Define the hyper-parameters 
hyper_parameters = {
    'coupling_layers': [10],
    'learning_rate': [0.001],
    "hidden_units": [256],
    "batch_size": [512]
}
grid = list(ParameterGrid(hyper_parameters))

Train the Real NVP model.

In [20]:
# Store hyper-parameters and validation loss
results_list = []

# Input data dimension
D = 20   

# 
for idx, hyper_parameters in enumerate(grid):
    D = dataset_train.data.shape[1]
        # Splits input dimensions for data partitions  
    D_half = D // 2
    if D % 2 == 0:
        input_dims = output_dims = D_half
    else:
        input_dims = D_half + 1
        output_dims = D_half

for idx, params in enumerate(grid):

    training_loader = DataLoader(dataset_train, batch_size=params['batch_size'], shuffle=False)
    val_loader = DataLoader(dataset_val, batch_size=params['batch_size'], shuffle=False)
    #test_loader = DataLoader(dataset_test, batch_size=params['batch_size'], shuffle=False)

    # Scaling (s) neural networks
    scaling_nn = lambda: nn.Sequential(
        nn.Linear(input_dims, params['hidden_units']),
        nn.LeakyReLU(),
        nn.Linear(params['hidden_units'], params['hidden_units']),
        nn.LeakyReLU(),
        nn.Linear(params['hidden_units'], output_dims),
        nn.Tanh()
    )

    # Shifting (t) neural networks
    shifting_nn = lambda: nn.Sequential(
        nn.Linear(input_dims, params['hidden_units']),
        nn.LeakyReLU(),
        nn.Linear(params['hidden_units'], params['hidden_units']),
        nn.LeakyReLU(),
        nn.Linear(params['hidden_units'], params['hidden_units']), nn.LeakyReLU(),
        nn.Linear(params['hidden_units'], output_dims)
    )
    
    
    # Base distribution set to multivariate standard Normal 
    z_dist = torch.distributions.MultivariateNormal(torch.zeros(D), torch.eye(D))
    
    # Real NVP model 
    model = density.DensityEstimation(D, z_dist, hyper_parameters['coupling_layers'], shifting_nn, scaling_nn)
    optimizer = torch.optim.Adamax([p for p in model.parameters() if p.requires_grad == True], lr=hyper_parameters['learning_rate'])

    # Directory to store the model weights and the density values
    save_dir = "./Results-P1-Monocytes/"
    
    os.makedirs(save_dir, exist_ok=True)
    
    # save_path = os.path.join(save_dir, model_name)
    # torch.save(model, save_path)
    
    name = os.path.join(save_dir, "P1_monocytes_Real_NVP_runtime")

    # Number of epochs for training
    num_epochs = 500

    # Maximum patience for early stopping
    max_patience = 15 

    # Initializes the Training class
    training_instance = Training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader, D)

    # Trains the model
    nll_val, epochs_trained = training_instance.train()
    
    # Number of epochs need to complete training
    print(f'Training completed after {epochs_trained} epochs.')

    results_list.append({
        'params': hyper_parameters,
        #'test_loss': test_loss,
        "val_loss": nll_val,
        #"epochs_needed": epochs_needed
    })


Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Loss improved, new model is saved.
Training completed after 44 epochs.


Evaluate the probability density function at every data point (cell) of the P1-Mono dataset.

In [23]:
# Input data dimension
D = 20   

# Real NVP model directory
models_directory = "./Results-P1-Monocytes/"


# Save density values at the corresponding model directory
results_directory = "./Results-P1-Monocytes/"

if not os.path.exists(results_directory):
    os.makedirs(results_directory)


for model_file in os.listdir(models_directory):
    
    # Iterate over available Real NVP models
    if model_file.endswith(".model"):  
        model_path = os.path.join(models_directory, model_file)
        model = torch.load(model_path)
        model.eval()
        
        # Load full data to evaluate the log-probability density function
        data_for_pdf = pd.read_csv("../Pre-processed-datasets/P1_monocytes.csv")
        data_for_pdf = data_for_pdf.iloc[:, :D].to_numpy()
        data_for_pdf = torch.tensor(data_for_pdf[:, :].astype(np.float32))


        # Create DataLoader
        evalLoader = DataLoader(data_for_pdf, batch_size=data_for_pdf.shape[0], shuffle=False)

        # Evaluate the model
        for indx_batch, batch in enumerate(evalLoader):
            output_pdf = model.log_probability_outputs(batch)
        
        output_pdf_arr = output_pdf.detach().numpy()
        # Negative log-likelihood values
        nll_results = pd.DataFrame(output_pdf_arr, columns=["pdf"])
        
        # Save results under the RESULTS directory based on the model name
        model_name = os.path.splitext(model_file)[0]  # Extract model name without extension
        save_path = os.path.join(results_directory, f"density_P1_monocytes.csv")
        nll_results.to_csv(save_path, index=False)
        print(f"Results saved for model: {model_name}")

C:\Users\Margarita\AppData\Local\Temp\ipykernel_43260\2017810115.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


Results saved for model: P1_monocytes_Real_NVP_runtime


In [24]:
# Negative log-likelihood values for the density weighting scheme in Tutorial 2
nll_results

,pdf
0,-19.902372
1,-19.294424
2,-17.888763
3,-26.869480
4,-24.836872
...,...
39251,-19.336288
39252,-23.642403
39253,-25.377218
39254,-26.672804


In [22]:
# Print hyper-parameters used and validation loss at each epoch
results_list

[{'params': {'batch_size': 512,
   'coupling_layers': 10,
   'hidden_units': 256,
   'learning_rate': 0.001},
  'val_loss': array([-18.42873789, -20.53990795, -21.35018664, -21.95961977,
         -22.42128629, -22.77094327, -23.07466864, -23.31068644,
         -23.42667563, -23.53709735, -23.58274717, -23.63933787,
         -23.65756715, -23.59989797, -23.66579537, -23.84350221,
         -24.01235373, -24.11671415, -24.12388113, -24.16444314,
         -24.18479281, -24.09253261, -23.92645803, -23.88138074,
         -23.84812994, -23.96849881, -24.15503353, -24.27462453,
         -24.33047452, -24.28720035, -24.26129507, -23.9720114 ,
         -24.13711108, -24.19782041, -24.24002987, -24.1581668 ,
         -24.09126423, -23.75038844, -23.94719572, -24.05157753,
         -24.14152668, -24.25532117, -24.28966812, -24.24060996,
         -24.22942244])}]